In [ ]:
%pip install pandas numpy matplotlib scikit-learn

Customer Reviews Classification & Customer Segmentation

Στόχοι
1) Μετατροπή της κλίμακας rating (1–5) σε 3 κατηγορίες (Bad / Mediocre / Good) και εκπαίδευση μοντέλων που προβλέπουν την κατηγορία από το κείμενο της κριτικής.
2) Καθαρισμός/μηχανική χαρακτηριστικών για το dataset πελατών και ομαδοποίηση με K-Means για υποστήριξη στοχευμένου marketing.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


# Customer Reviews Classification & Customer Segmentation

## Εισαγωγή

Η παρούσα εργασία στοχεύει στην ανάλυση δεδομένων ηλεκτρονικού εμπορίου με χρήση τεχνικών μηχανικής μάθησης.
Αρχικά, μελετάται το περιεχόμενο κριτικών πελατών και η μετατροπή της αριθμητικής βαθμολογίας σε κατηγορίες
συναισθήματος. Στη συνέχεια, πραγματοποιείται ομαδοποίηση πελατών με σκοπό την υποστήριξη στοχευμένων
στρατηγικών marketing.


In [ ]:
items = pd.read_csv("data/20191226-items.csv")
reviews = pd.read_csv("data/20191226-reviews.csv")
customers = pd.read_csv("data/customers.csv")

items.head(), reviews.head(), customers.head()


    ## 1. Έλεγχος δομής & ποιότητας δεδομένων
Σε αυτό το βήμα ελέγχουμε τύπους μεταβλητών, ελλείπουσες τιμές και βασικές περιγραφικές πληροφορίες ώστε να σχεδιάσουμε σωστά την προεπεξεργασία.


In [ ]:
print("ITEMS:", items.shape)
display(items.head(3))
display(items.isna().sum().sort_values(ascending=False).head(10))

print("\nREVIEWS:", reviews.shape)
display(reviews.head(3))
display(reviews.isna().sum().sort_values(ascending=False).head(10))

print("\nCUSTOMERS:", customers.shape)
display(customers.head(3))
display(customers.isna().sum().sort_values(ascending=False).head(10))


## 2. Συνένωση (Join) των datasets & δημιουργία της μεταβλητής Cat_Rating
Ενώνουμε τα datasets βάσει του κοινού κλειδιού `asin`. Στη συνέχεια μετατρέπουμε τη βαθμολογία 1–5 σε 3 κατηγορίες:
- 1–2: Bad
- 3: Mediocre
- 4–5: Good


In [ ]:
# Join reviews με επιλεγμένα πεδία από items (ώστε να κρατάμε και πληροφορίες προϊόντος)
df = reviews.merge(
    items[["asin", "brand", "title", "price", "rating", "totalReviews"]],
    on="asin",
    how="left",
    suffixes=("_review", "_item")
)

print(df.shape)
display(df.head(3))


In [ ]:
print("Columns in reviews:")
print(reviews.columns.tolist())

print("\nColumns in items:")
print(items.columns.tolist())

print("\nColumns in df (after merge):")
print(df.columns.tolist())


In [ ]:
# Βρες αυτόματα ποια στήλη είναι η βαθμολογία της κριτικής
possible_rating_cols = ["rating", "Rating", "stars", "score", "overall", "reviewRating", "rating_review"]
rating_col = next((c for c in possible_rating_cols if c in df.columns), None)

if rating_col is None:
    # Αν δεν τη βρει, προσπάθησε να βρει οποιαδήποτε στήλη που περιέχει τη λέξη 'rating'
    candidates = [c for c in df.columns if "rating" in c.lower()]
    raise ValueError(f"Δεν βρέθηκε στήλη rating. Πιθανές στήλες: {candidates}")

print("✅ Χρησιμοποιώ ως rating τη στήλη:", rating_col)

# Μετατροπή σε numeric
df[rating_col] = pd.to_numeric(df[rating_col], errors="coerce")

# Δημιουργία Cat_Rating
def to_cat(r):
    if pd.isna(r):
        return np.nan
    if r <= 2:
        return "Bad"
    elif r == 3:
        return "Mediocre"
    else:
        return "Good"

df["Cat_Rating"] = df[rating_col].apply(to_cat)

display(df[["asin", rating_col, "Cat_Rating"]].head(10))
display(df["Cat_Rating"].value_counts(dropna=False))



In [ ]:
counts = df["Cat_Rating"].value_counts()

plt.figure()
counts.plot(kind="bar")
plt.title("Κατανομή κατηγοριών Cat_Rating")
plt.xlabel("Cat_Rating")
plt.ylabel("Πλήθος")
plt.show()


## 3. Προετοιμασία Κειμένου για Classification

Στο στάδιο αυτό ορίζονται:
- ως μεταβλητή εισόδου (X) το κείμενο της κριτικής
- ως μεταβλητή στόχου (y) η κατηγορία συναισθήματος Cat_Rating


In [ ]:
# Δείξε όλες τις στήλες του df
print(df.columns.tolist())


In [ ]:
# Βρες αυτόματα τη στήλη που περιέχει το κείμενο της κριτικής
possible_text_cols = ["reviewText", "review", "text", "content", "summary", "body"]

text_col = next((c for c in possible_text_cols if c in df.columns), None)

if text_col is None:
    # Αν δεν βρεθεί, δοκίμασε να εντοπίσεις στήλες που μοιάζουν
    candidates = [c for c in df.columns if any(k in c.lower() for k in ["review", "text", "content", "summary", "body"])]
    raise ValueError(f"❌ Δεν βρέθηκε στήλη κειμένου. Πιθανές στήλες: {candidates}")

print("✅ Χρησιμοποιώ ως κείμενο τη στήλη:", text_col)

# Dataset για classification
df_clf = df[[text_col, "Cat_Rating"]].dropna()

print("Shape df_clf:", df_clf.shape)
df_clf.head()


In [ ]:
# 3.2 Ορισμός X (κείμενο) και y (Cat_Rating)
X = df_clf[text_col]
y = df_clf["Cat_Rating"]

print("Πλήθος δειγμάτων:", X.shape[0])
print(y.value_counts())


### 3.3 Διαχωρισμός σε Training / Test set (Stratified Split)
Διατηρούμε την ίδια αναλογία κατηγοριών (Bad/Mediocre/Good) σε train και test.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))
print("\nTrain distribution:\n", y_train.value_counts(normalize=True))
print("\nTest distribution:\n", y_test.value_counts(normalize=True))


### 3.4 TF-IDF Μετατροπή Κειμένου σε Χαρακτηριστικά
Χρησιμοποιούμε TF-IDF με unigrams + bigrams και stopwords (English) για καλύτερη αναπαράσταση κειμένου.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words="english",
    lowercase=True,
    max_features=20000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF train shape:", X_train_tfidf.shape)
print("TF-IDF test shape:", X_test_tfidf.shape)


## 4. Μοντέλο 1: Decision Tree
Εκπαιδεύουμε Decision Tree πάνω στα TF-IDF χαρακτηριστικά και αξιολογούμε με classification report & confusion matrix.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

tree = DecisionTreeClassifier(
    random_state=42,
    max_depth=30,
    min_samples_leaf=2
)

tree.fit(X_train_tfidf, y_train)
y_pred_tree = tree.predict(X_test_tfidf)

print("Decision Tree - Classification Report:\n")
print(classification_report(y_test, y_pred_tree))

cm = confusion_matrix(y_test, y_pred_tree, labels=["Bad", "Mediocre", "Good"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Bad", "Mediocre", "Good"])
disp.plot()
plt.title("Decision Tree - Confusion Matrix")
plt.show()


## 5. Μοντέλο 2: KNN (δοκιμή διαφορετικών k)
Δοκιμάζουμε διαφορετικά k και επιλέγουμε το καλύτερο βάσει Accuracy στο test set.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

k_values = [1, 3, 5, 7, 9, 11, 15, 21]
accs = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, weights="distance")
    knn.fit(X_train_tfidf, y_train)
    preds = knn.predict(X_test_tfidf)
    acc = accuracy_score(y_test, preds)
    accs.append(acc)
    print(f"k={k} -> accuracy={acc:.4f}")

plt.figure()
plt.plot(k_values, accs, marker="o")
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.title("KNN: Accuracy vs k")
plt.show()

best_k = k_values[int(np.argmax(accs))]
print("\n✅ Best k based on accuracy:", best_k)


In [ ]:
best_k = 7


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

knn_best = KNeighborsClassifier(n_neighbors=best_k, weights="distance")
knn_best.fit(X_train_tfidf, y_train)

y_pred_knn = knn_best.predict(X_test_tfidf)

print(f"KNN (k={best_k}) Classification Report:\n")
print(classification_report(y_test, y_pred_knn))

cm = confusion_matrix(y_test, y_pred_knn, labels=["Bad", "Mediocre", "Good"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Bad", "Mediocre", "Good"])
disp.plot()
plt.title(f"KNN (k={best_k}) - Confusion Matrix")
plt.show()


Σύγκριση Decision Tree vs KNN
Συγκρίνουμε τα μοντέλα με Accuracy και Macro F1-score (λόγω ανισορροπίας κλάσεων).

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Decision Tree predictions: y_pred_tree
# KNN predictions: y_pred_knn

acc_tree = accuracy_score(y_test, y_pred_tree)
f1_tree  = f1_score(y_test, y_pred_tree, average="macro")

acc_knn = accuracy_score(y_test, y_pred_knn)
f1_knn  = f1_score(y_test, y_pred_knn, average="macro")

print("=== Σύγκριση Μοντέλων ===")
print(f"Decision Tree: accuracy={acc_tree:.4f}, macroF1={f1_tree:.4f}")
print(f"KNN (k={best_k}): accuracy={acc_knn:.4f}, macroF1={f1_knn:.4f}")


## 6. Ομαδοποίηση Πελατών (Customer Segmentation)


## 6. Customer Segmentation με K-Means


### 6.1 Φόρτωση και αρχική εξερεύνηση δεδομένων πελατών


In [ ]:
# 6.1 Φόρτωση δεδομένων πελατών

customers = pd.read_csv("data/customers.csv")

# Βασική επισκόπηση
customers.shape


### Οπτικοποίηση Κατηγορικών Μεταβλητών

Στο παρακάτω στάδιο παρουσιάζονται γραφήματα για βασικές κατηγορικές μεταβλητές
(Εκπαίδευση και Οικογενειακή Κατάσταση), ώστε να γίνει κατανοητή η κατανομή
των πελατών πριν την προεπεξεργασία και την ομαδοποίηση.


In [ ]:
import matplotlib.pyplot as plt

# Κατανομή Εκπαίδευσης
edu_counts = customers["Education"].value_counts()

plt.figure(figsize=(6,4))
plt.bar(edu_counts.index, edu_counts.values)
plt.title("Κατανομή Εκπαίδευσης Πελατών")
plt.xticks(rotation=45)
plt.ylabel("Πλήθος")
plt.show()

# Κατανομή Οικογενειακής Κατάστασης
mar_counts = customers["Marital_Status"].value_counts()

plt.figure(figsize=(6,4))
plt.bar(mar_counts.index, mar_counts.values)
plt.title("Κατανομή Οικογενειακής Κατάστασης Πελατών")
plt.xticks(rotation=45)
plt.ylabel("Πλήθος")
plt.show()


“6.2 Μηχανική Δεδομένων & Καθαρισμός

In [ ]:
import pandas as pd
import numpy as np

# Δουλεύουμε σε αντίγραφο
cust = customers.copy()

# 1) Μετατροπή Dt_Customer σε ημερομηνία (για να πάρουμε reference year)
cust["Dt_Customer"] = pd.to_datetime(cust["Dt_Customer"], dayfirst=True, errors="coerce")
ref_year = int(cust["Dt_Customer"].dt.year.max())  # π.χ. 2014

# 2) Ηλικία
cust["Age"] = ref_year - cust["Year_Birth"]

# 3) Παιδιά (Kidhome + Teenhome)
cust["Children"] = cust["Kidhome"] + cust["Teenhome"]

# 4) Δαπάνες (σύνολο σε όλες τις κατηγορίες)
spend_cols = ["MntWines","MntFruits","MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds"]
cust["Spending"] = cust[spend_cols].sum(axis=1)

# 5) Καθαρισμός Οικογενειακής Κατάστασης
# - σβήνουμε σπάνιες κατηγορίες (threshold=10 μπορείς να το αλλάξεις)
vc = cust["Marital_Status"].value_counts()
rare = vc[vc < 10].index
cust = cust[~cust["Marital_Status"].isin(rare)].copy()

# - Married + Together -> Couple
cust["Marital_Status_Clean"] = cust["Marital_Status"].replace({
    "Married": "Couple",
    "Together": "Couple"
})

# 6) Εκπαίδευση σε 3 κατηγορίες
cust["Education_3"] = cust["Education"].replace({
    "Basic": "Basic",
    "2n Cycle": "Basic",
    "Graduation": "Graduate",
    "Master": "Postgrad",
    "PhD": "Postgrad"
})

# 7) Missing values (Income έχει συνήθως NaN)
# Επιλογή: συμπλήρωση με median
cust["Income"] = cust["Income"].fillna(cust["Income"].median())

# Έλεγχος ότι δεν έχουμε NaN
cust.isna().sum().sort_values(ascending=False).head(10)


In [ ]:
import matplotlib.pyplot as plt

# Education (3 categories)
plt.figure(figsize=(7,4))
cust["Education_3"].value_counts().plot(kind="bar")
plt.title("Κατανομή Εκπαίδευσης (3 κατηγορίες)")
plt.ylabel("Πλήθος")
plt.xticks(rotation=45)
plt.show()

# Marital_Status (clean)
plt.figure(figsize=(7,4))
cust["Marital_Status_Clean"].value_counts().plot(kind="bar")
plt.title("Κατανομή Οικογενειακής Κατάστασης (καθαρισμένη)")
plt.ylabel("Πλήθος")
plt.xticks(rotation=45)
plt.show()


**Σχόλιο κατηγορικών μεταβλητών**

Από τα παραπάνω γραφήματα παρατηρούμε ότι:
- Η πλειονότητα των πελατών έχει επίπεδο εκπαίδευσης *Graduation*, ενώ λιγότεροι πελάτες ανήκουν στις κατηγορίες *PhD* και *Master*.
- Στην οικογενειακή κατάσταση κυριαρχούν οι κατηγορίες *Married/Together* και *Single*, ενώ ορισμένες κατηγορίες εμφανίζουν πολύ μικρό πλήθος.

Οι παρατηρήσεις αυτές δικαιολογούν τη συγχώνευση ή/και αφαίρεση σπάνιων κατηγοριών στο επόμενο στάδιο προεπεξεργασίας πριν την εφαρμογή του K-Means.


In [ ]:
customers = pd.read_csv("data/customers.csv", sep="\t")

customers.shape


In [ ]:
customers.head()


In [ ]:
# Αντιγραφή dataset για καθαρισμό
cust_clean = customers.copy()

# Στήλες που δεν βοηθούν στο clustering
drop_cols = ["ID", "Dt_Customer"]
cust_clean = cust_clean.drop(columns=drop_cols)

cust_clean.head()


In [ ]:
cust_clean.isna().sum().sort_values(ascending=False)


In [ ]:
cust["Income"] = cust["Income"].fillna(
    cust["Income"].median()
)

cust.isna().sum().max()


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Κατηγορικές μεταβλητές (καθαρισμένες)
cat_cols = ["Education_3", "Marital_Status_Clean"]

# Αριθμητικές μεταβλητές
num_cols = cust.drop(columns=cat_cols).select_dtypes(include=np.number).columns

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(drop="first"), cat_cols)
    ]
)

# Τελικός πίνακας για K-Means
X_kmeans = preprocessor.fit_transform(cust)

X_kmeans.shape


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Διάλεξε 4-5 σημαντικά χαρακτηριστικά (όπως λέει η εκφώνηση)
pair_cols = ["Age", "Income", "Spending", "Children", "Recency"]

df_pair = cust[pair_cols].dropna().sample(600, random_state=42)  # sample για να μη βαραίνει

pd.plotting.scatter_matrix(df_pair, figsize=(10, 10))
plt.suptitle("Pair Plot (Scatter Matrix) σημαντικών χαρακτηριστικών", y=1.02)
plt.show()


In [ ]:
inertias = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_kmeans)
    inertias.append(kmeans.inertia_)

plt.figure()
plt.plot(k_range, inertias, marker="o")
plt.xlabel("Αριθμός clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method για επιλογή k")
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

sil_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_kmeans)
    score = silhouette_score(X_kmeans, labels)
    sil_scores.append(score)
    print(f"k={k} → silhouette={score:.4f}")

# Γράφημα
plt.figure()
plt.plot(k_range, sil_scores, marker="o")
plt.xlabel("Αριθμός clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score vs k")
plt.show()


In [ ]:
from sklearn.cluster import KMeans

k_final = 5
kmeans_final = KMeans(n_clusters=k_final, random_state=42, n_init=10)

clusters = kmeans_final.fit_predict(X_kmeans)

cust_clean["Cluster"] = clusters
cust_clean["Cluster"].value_counts()


In [ ]:
cluster_profile = cust_clean.groupby("Cluster").mean(numeric_only=True)
cluster_profile


In [ ]:
cluster_profile.round(2)


### Ερμηνεία Ομάδων Πελατών

- **Cluster 0**: Πελάτες χαμηλότερου εισοδήματος με μικρές αγορές σε όλα τα προϊόντα.
- **Cluster 1**: Υψηλού εισοδήματος πελάτες με πολύ υψηλή κατανάλωση κρασιών και κρέατος.
- **Cluster 2**: Μεσαίου εισοδήματος, ισορροπημένες αγορές σε πολλά προϊόντα.
- **Cluster 3**: Πελάτες με παιδιά στο σπίτι και χαμηλή συνολική κατανάλωση.
- **Cluster 4**: Πελάτες μέσης ηλικίας με επιλεκτικές αγορές (π.χ. κρασιά).
